In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [12]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

In [17]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [18]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [19]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [20]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 140us/sample - loss: 1.8388 - val_loss: 0.7987
Epoch 2/100
7740/7740 [==============================] - 0s 33us/sample - loss: 0.6962 - val_loss: 0.6497
Epoch 3/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.6200 - val_loss: 0.6131
Epoch 4/100
7740/7740 [==============================] - 0s 33us/sample - loss: 0.5833 - val_loss: 0.5691
Epoch 5/100
7740/7740 [==============================] - 0s 33us/sample - loss: 0.5617 - val_loss: 0.5544
Epoch 6/100
7740/7740 [==============================] - 0s 32us/sample - loss: 0.5332 - val_loss: 0.5281
Epoch 7/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.5323 - val_loss: 0.5171
Epoch 8/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.5034 - val_loss: 0.4999
Epoch 9/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.4954 - val_loss: 0.49

7740/7740 [==============================] - 0s 31us/sample - loss: 0.3436 - val_loss: 0.3505
Epoch 78/100
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3461 - val_loss: 0.3494
Epoch 79/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3436 - val_loss: 0.3493
Epoch 80/100
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3417 - val_loss: 0.3477
Epoch 81/100
7740/7740 [==============================] - 0s 32us/sample - loss: 0.3447 - val_loss: 0.3472
Epoch 82/100
7740/7740 [==============================] - 0s 32us/sample - loss: 0.3451 - val_loss: 0.3482
Epoch 83/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3411 - val_loss: 0.3469
Epoch 84/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3426 - val_loss: 0.3459
Epoch 85/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3403 - val_loss: 0.3450
Epoch 86/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 31us/sample - loss: 0.3630 - val_loss: 0.3650
Epoch 53/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3621 - val_loss: 0.3642
Epoch 54/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3616 - val_loss: 0.3625
Epoch 55/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3608 - val_loss: 0.3623
Epoch 56/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3597 - val_loss: 0.3617
Epoch 57/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3594 - val_loss: 0.3618
Epoch 58/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.3583 - val_loss: 0.3600
Epoch 59/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.3580 - val_loss: 0.3596
Epoch 60/100
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3571 - val_loss: 0.3588
Epoch 61/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 31us/sample - loss: 0.3826 - val_loss: 0.3909
Epoch 28/100
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3803 - val_loss: 0.3894
Epoch 29/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3781 - val_loss: 0.3868
Epoch 30/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3759 - val_loss: 0.3853
Epoch 31/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3741 - val_loss: 0.3837
Epoch 32/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3723 - val_loss: 0.3821
Epoch 33/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3699 - val_loss: 0.3806
Epoch 34/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3686 - val_loss: 0.3796
Epoch 35/100
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3672 - val_loss: 0.3786
Epoch 36/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 28us/sample - loss: 0.8206 - val_loss: 0.7385
Epoch 3/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6759 - val_loss: 0.6621
Epoch 4/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.6226 - val_loss: 0.6241
Epoch 5/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5919 - val_loss: 0.5959
Epoch 6/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5701 - val_loss: 0.5736
Epoch 7/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.5497 - val_loss: 0.5564
Epoch 8/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.5359 - val_loss: 0.5416
Epoch 9/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5217 - val_loss: 0.5286
Epoch 10/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.5121 - val_loss: 0.5172
Epoch 11/100
7740/7740 [==============================] -

...

3870/3870 [==============================] - 0s 9us/sample - loss: 0.5813
Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 0s 39us/sample - loss: 3.0266 - val_loss: 1.6088
Epoch 2/100
7740/7740 [==============================] - 0s 29us/sample - loss: 1.2755 - val_loss: 1.0115
Epoch 3/100
7740/7740 [==============================] - 0s 30us/sample - loss: 0.9163 - val_loss: 0.8506
Epoch 4/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.8059 - val_loss: 0.7851
Epoch 5/100
7740/7740 [==============================] - 0s 27us/sample - loss: 0.7553 - val_loss: 0.7478
Epoch 6/100
7740/7740 [==============================] - 0s 30us/sample - loss: 0.7253 - val_loss: 0.7226
Epoch 7/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.7030 - val_loss: 0.7025
Epoch 8/100
7740/7740 [==============================] - 0s 27us/sample - loss: 0.6850 - val_loss: 0.6854
Epoch 9/100
7740/7740 [=======

7740/7740 [==============================] - 0s 28us/sample - loss: 0.4269 - val_loss: 0.4284
Epoch 76/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4262 - val_loss: 0.4277
Epoch 77/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4254 - val_loss: 0.4270
Epoch 78/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4245 - val_loss: 0.4258
Epoch 79/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4238 - val_loss: 0.4252
Epoch 80/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4231 - val_loss: 0.4246
Epoch 81/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4222 - val_loss: 0.4236
Epoch 82/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4216 - val_loss: 0.4229
Epoch 83/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4209 - val_loss: 0.4222
Epoch 84/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 28us/sample - loss: 0.4735 - val_loss: 0.4685
Epoch 51/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4719 - val_loss: 0.4674
Epoch 52/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4702 - val_loss: 0.4655
Epoch 53/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4689 - val_loss: 0.4642
Epoch 54/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4679 - val_loss: 0.4638
Epoch 55/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4661 - val_loss: 0.4620
Epoch 56/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4653 - val_loss: 0.4608
Epoch 57/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4639 - val_loss: 0.4597
Epoch 58/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4626 - val_loss: 0.4587
Epoch 59/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 28us/sample - loss: 0.5466 - val_loss: 0.5489
Epoch 26/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5402 - val_loss: 0.5427
Epoch 27/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5345 - val_loss: 0.5369
Epoch 28/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5289 - val_loss: 0.5314
Epoch 29/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5236 - val_loss: 0.5263
Epoch 30/100
7740/7740 [==============================] - 0s 27us/sample - loss: 0.5184 - val_loss: 0.5215
Epoch 31/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5138 - val_loss: 0.5167
Epoch 32/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5092 - val_loss: 0.5123
Epoch 33/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5049 - val_loss: 0.5079
Epoch 34/100
7740/7740 [==========================

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f3d004e5748>, as the constructor either does not set or modifies parameter learning_rate

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
model = rnd_search_cv.best_estimator_.model